# StatsPipeline & Plotting Example

<div class="alert alert-block alert-info">
    
This example shows how to set up a statistical analysis pipeline using `StatsPipeline`. 

Additionally, it demonstrates the new plotting functions of `BioPsyKit`, that wrap the `boxplot()` function of `seaborn` and offer additional, often used features, such as adding significance brackets:
    
* `biopsykit.plotting.feature_boxplot()` and `biopsykit.plotting.multi_feature_boxplot()`
* as well as the derived functions specialized for saliva features: `biopsykit.protocols.plotting.saliva_feature_boxplot()` and `biopsykit.protocols.plotting.saliva_multi_feature_boxplot()`.
</div>

## Import and Helper Functions

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Data Import

In [ ]:
sample_times = [-30, -1, 30, 40, 50, 60, 70]

condition_order = ["Control", "Intervention"]

### Raw Cortisol

In [ ]:
cort_samples = bp.example_data.get_saliva_example()
cort_samples.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
# specialized function for plotting saliva data
bp.protocols.plotting.saliva_plot(
    data=cort_samples, 
    saliva_type="cortisol", 
    sample_times=sample_times, 
    test_times=[0, 30], 
    sample_times_absolute=True,
    test_title="TEST",
    ax=ax
)

### Compute Cortisol Features

In [ ]:
auc = bp.saliva.auc(cort_samples, saliva_type="cortisol", sample_times=sample_times, compute_auc_post=True, remove_s0=True)
max_inc = bp.saliva.max_increase(cort_samples, saliva_type="cortisol", remove_s0=True)
slope = bp.saliva.slope(cort_samples, sample_idx=[1, 4], sample_times=sample_times, saliva_type="cortisol")

cort_features = pd.concat([auc, max_inc, slope], axis=1)
cort_features = bp.saliva.utils.saliva_feature_wide_to_long(cort_features, saliva_type="cortisol")
cort_features.head()

## General Information

The purpose of creating such a statistical analysis pipeline is to assemble several steps of a typical statistical analysis procedure while setting different parameters. The parameters passed to this class depend on the used statistical functions.

The interface of this class is inspired by the scikit-learn [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for machine learning tasks.


The different steps of statistical analysis can be divided into categories:

* *Preparatory Analysis* (``prep``): Analyses applied to the data before performing the actual statistical analysis, such as:
    * ``normality``: Testing whether a random sample comes from a normal distribution.
    * ``equal_var``: Testing the equality of variances (homoscedasticity).
* *Statistical Tests* (``test``): Statistical test to determine differences or similarities in the data, such as:
    * ``pairwise_ttests``: Pairwise T-tests (either for independent or dependent samples).
    * ``anova``: One-way or N-way ANOVA.
    * ``welch_anova``: One-way Welch-ANOVA.
    * ``rm_anova``: One-way and two-way repeated measures ANOVA.
    * ``mixed_anova``: Mixed-design (split-plot) ANOVA.
    * ``kruskal``: Kruskal-Wallis H-test for independent samples.
* *Posthoc Tests* (``posthoc``): Posthoc tests to determine differences of individual groups if more than two
  groups are analyzed, such as:
    * ``pairwise_ttests``: Pairwise T-tests (either for independent or dependent samples).
    * ``pairwise_tukey``: Pairwise Tukey-HSD post-hoc test.
    * ``pairwise_gameshowell``: Pairwise Games-Howell post-hoc test.

A ``StatsPipeline`` consists of a list of tuples specifying the individual ``steps`` of the pipeline.
The first value of each tuple indicates the category this step belongs to (``prep``, ``test``, or ``posthoc``),
the second value indicates the analysis function to use in this step (e.g., ``normality``, or ``rm_anova``).

Furthermore, a ``params`` dictionary specifying the parameters and variables for statistical analysis
needs to be supplied. Parameters can either be specified *globally*, i.e., for all steps in the pipeline
(the default), or *locally*, i.e., only for one specific category, by prepending the category name and separating it from the parameter name by a `__`. The parameters depend on the type of analysis used in the pipeline. 


Examples are:

* ``dv``: column name of the dependent variable
* ``between``: column name of the between-subject factor
* ``within``: column name of the within-subject factor
* ``effsize``: type of effect size to compute (if applicable)
* ``multicomp``: whether (and how) to apply multi-comparison correction of p-values to the *last* step in the
  pipeline (either "test" or "posthoc") using [``StatsPipeline.multicomp()``](https://biopsykit.readthedocs.io/en/latest/api/biopsykit.stats.html#biopsykit.stats.StatsPipeline.multicomp). The arguments are supplied via dictionary.
* ...

## T-Tests

In [ ]:
cort_features.head()

`cort_features` contains multiple features that need to be analyzed individually. The analysis pipeline can be applied to each feature individually by specifying a column to group the dataframe by (`groupby` parameter). The result dataframes will then contain the `groupby` column as index level.

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "groupby": "saliva_feature",
        "between": "condition"
    }
)

pipeline.apply(cort_features);

### Display Results

In [ ]:
# display all results
pipeline.display_results()
# only significant results
# pipeline.display_results(sig_only=True)
# only significant results from the "posthoc" category (results from other categories will all be displayed)
# pipeline.display_results(sig_only="posthoc")

### Further functions of ``StatsPipeline``

In [ ]:
# analysis categories and their respective analysis steps
print(pipeline.category_steps)
# dictionary with analysis results per step
results = pipeline.results
# get results from normality check
display(results["normality"])
# return only results from one dategory
display(pipeline.results_cat("posthoc"))
# export the whole pipeline as Excel sheet
# pipeline.export_statistics("path_to_file.xlsx")

## Mixed ANOVA

In [ ]:
# construct some example to demonstrate mixed-anova analysis
data_example = multi_xs(cort_samples, ["2", "3"], level="sample")

In [ ]:
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "mixed_anova"),
        ("posthoc", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "between": "condition",
        "within": "sample",
        "subject": "subject",
        "padjust": "bonf" # specify multicorrection method to be applied on the posthoc tests
    }
)

pipeline.apply(data_example)
pipeline.display_results()

## Repeated-Measurements ANOVA

In [ ]:
data_slice = data_example.xs("Control", level="condition")

pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "rm_anova"),
        ("posthoc", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "within": "sample",
        "subject": "subject"
    }
)

pipeline.apply(data_slice)
pipeline.display_results()

## Get Significance Brackets from `StatsPipeline`

`StatsPipeline.sig_brackets()` returns the significance brackets and the corresponding p-values to add to the plotting functions of `BioPsyKit`.

The method takes the following parameters (from the documentation):
* `stats_category_or_data`: either a string of the pipeline category to use for generating significance brackets or a dataframe with statistical if significance brackets should be generated from the dataframe
* `stats_type`: type of analysis performed ("between", "within", "interaction"). Needed to extract the correct information from the analysis dataframe
* `plot_type`: type of plot for which significance brackets are generated: "multi" if boxplots are grouped (by a ``hue`` variable), "single" (the default) otherwise
* `features`: feature(s) displayed in the boxplot. The resulting significance brackets will be filtered accordingly to only contain features present in the boxplot. It can have the following formats:
    * ``str``: only one feature is plotted in the boxplot  
      => returns significance brackets of only one feature
    * ``list``: multiple features are combined into *one* `Axes` object (i.e., no subplots)  
      => returns significance brackets of multiple features
    * ``dict``: if boxplots of features are organized in subplots then `features` needs to dictionary with the feature (or list of features) per subplot (``subplots`` is ``True``)  
      => returns dictionary with significance brackets per subplot
* `x`: name of column used as `x` axis in the boxplot. Only required if `plot_type` is "multi".
     

### Single Plot – One Feature

This example shows how to add significance brackets to a *single plot* with a *single boxplot* where only one type of feature is plotted (e.g., only to display `max_inc` feature, where the different groups are separated by the `x` variable). 

If the feature to be plotted is only a subset of different features, it must be filtered via the `features` parameter:

In [ ]:
# rerun the t-test pipeline for plotting
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "groupby": "saliva_feature",
        "between": "condition"
    }
)

pipeline.apply(cort_features);

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets("test", stats_type="between", plot_type="single", features="max_inc")
print(box_pairs)

### Single Plot – Multiple Features

This example shows how to add significance brackets to a *single plot* with *multiple boxplots* where multiple types of features are plotted (e.g., to display the `max_inc` and `slope14` features, where different features are separated by the `x` variable and the different groups are separated by the `hue` variable).

If only a subset of features should be plotted, the features of interest must be filtered via the `features` parameter:

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=["max_inc", "slope14"]
)
print(box_pairs)

### Multiple Plots – Multiple Features

This example shows how to add significance brackets to *multiple subplots* with *multiple boxplots* per subplot (e.g., to display the `max_inc` and the `slope14` feature where `max_inc` and `slope14` each have their own subplot, the features are plotted along the `x` variable and the different groups are separated by the `hue` variable). For creating significance brackets to be used in subplots set the `subplots` parameter to `True`. 

By default, each feature is assumed to be in its **own subplot** (see the next example if you want to change the behavior).

If only a subset of features should be plotted, the features of interest must be filtered via the `features` parameter:

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=["max_inc", "slope14"],
    subplots=True
)
print(box_pairs)

If you want to specify a **custom mapping** of features to subplots you can provide a dictionary specifying this mapping as ``features`` argument (here, we want to have the features `auc_i` and `auc_g` in one subplot, and `max_inc` and `slope14` in another subplot):

In [ ]:
box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    "between", 
    "multi", 
    x="saliva_feature", 
    features={"auc": ["auc_i", "auc_g"], "inc": ["max_inc", "slope14"]}, 
    subplots=True
)
print(box_pairs)

## Plotting

### Single Plot – One Feature

This example shows how to plot a single feature in a single boxplot using `biopsykit.plotting.feature_boxplot`. The two conditions are plotted along the `x` axis.

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="single", 
    features=features
)

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="condition", 
    y="cortisol", 
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

### Single Plot – Multiple Features

This example is the *same* as the one in the **Single Plot – One Feature** example above, but this time, the (single) feature is plotted along the `x` axis and the two groups are separated by the `hue` parameter. This makes it `plot_type` "multi" and thus requires to specify the `x` parameter in `StatsPipeline.sig_brackets()`.

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=features
)

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

Now with a "real" example: We use `biopsykit.plotting.feature_boxplot` to plot actually multiple features along the `x` axis with the `hue` variable separating the conditions.

In this example, however, no feature shows statistically significant differences between the two conditions.

In [ ]:
features = ["auc_g", "auc_i"]
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    features=features, 
    plot_type="multi", 
    x="saliva_feature"
)

fig, ax = plt.subplots()
bp.plotting.feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

### Multiple Plots – Multiple Features

This example shows how to plot features in *multiple subplots* with *multiple boxplots* per subplot. Here, `max_inc` and the `slope14` features are displayed in their own subplot and the features `auc_i` and `auc_g` are combined into one joint subplot. The features are separated by the `x` variable and the different groups are separated by the `hue` variable.

This *custom mapping* of features to subplots can be provided via a dictionary passed to ``features`` argument.

In [ ]:
features = {
    "auc": ["auc_g", "auc_i"], 
    "max_inc": "max_inc", 
    "slope14": "slope14"
}

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=features, 
    subplots=True
)

data_plot = cort_features.copy()

bp.plotting.multi_feature_boxplot(
    data=data_plot, 
    x="saliva_feature", 
    y="cortisol", 
    hue="condition",
    group="saliva_feature",
    features=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }
)
fig.tight_layout()

### Use Specialized `saliva_feature_plot` functions

In addition to the "general-purpose" plotting functions `BioPsyKit` also offers specialized plotting functions for saliva features since plotting saliva data is a commonly performed task. These functions offer a better styling of axis and labels for saliva data.

In [ ]:
# rerun the pipeline
pipeline = StatsPipeline(
    steps=[
        ("prep", "normality"),
        ("prep", "equal_var"),
        ("test", "pairwise_ttests")
    ],
    params={
        "dv": "cortisol",
        "groupby": "saliva_feature",
        "between": "condition"
    }
)

pipeline.apply(cort_features)
pipeline.display_results()

#### Single Plot – One Feature

In [ ]:
features = "max_inc"
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    features=features, 
    plot_type="single"
)

fig, ax = plt.subplots()
bp.protocols.plotting.saliva_feature_boxplot(
    data=data_plot, 
    x="condition", 
    saliva_type="cortisol", 
    feature=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

#### Single Plot – Multiple Feature

In [ ]:
features = ["auc_g", "auc_i"]
data_plot = multi_xs(cort_features, features, level="saliva_feature")

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    features=features, 
    plot_type="multi", 
    x="saliva_feature"
)

fig, ax = plt.subplots()
bp.protocols.plotting.saliva_feature_boxplot(
    data=data_plot, 
    x="saliva_feature",
    saliva_type="cortisol", 
    hue="condition",
    feature=features,
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }, 
    ax=ax
)
fig.tight_layout()

#### Plot Multiple Features in Subplots

In [ ]:
features = {
    "auc": ["auc_g", "auc_i"], 
    "max_inc": "max_inc", 
    "slope14": "slope14"
}

box_pairs, pvalues = pipeline.sig_brackets(
    "test", 
    stats_type="between", 
    plot_type="multi", 
    x="saliva_feature", 
    features=features, 
    subplots=True
)

data_plot = cort_features.copy()

bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=data_plot, 
    saliva_type="cortisol", 
    features=features,
    hue="condition",
    stats_kwargs={
        'box_pairs': box_pairs, 
        'pvalues': pvalues
    }
)
fig.tight_layout()